# Using pyRAPL to measure energy consumption

Actually quite simple! Unfortunately, pyRAPL is not compatible with MacOS so this should be runned on a server. The code will then look something like this:

In [ ]:
import pyRAPL

def measurement(k, iterations, input_file, class_name, suppression, data_type, dataset): 
    # init settings pyRAPL
    pyRAPL.setup()
    csv_output = pyRAPL.outputs.CSVOutput("Energy_measurement.csv")

    @pyRAPL.measureit(output=csv_output, number=1)
    def Energy_consumption():
        # run our file, in this case ARX
        subprocess.run(['sudo', 'java', '-cp', '.:libraries/*', class_name, 
                        str(k), input_file, str(suppression), dataset], capture_output=False)

    # we wish to perform an n amount of measurements such that we can compute the weighted average
    for _ in range(iterations + 1):
        Energy_consumption()
        print(f"Measurement { _ + 1} out of {iterations} completed.")
    
    # save the measurements in our csv file
    csv_output.save()


if __name__ == '__main__':

    # perform measurement
    measurement(k, int(data["iterations"]), data["input_file"], data["class_name"], 
                        data["suppression_limit"], data["type"], data["data_set"])

## Then the output will look something like this

In [1]:
import pandas as pd

df = pd.read_csv('Data/Energy/Energy_nn.csv') # neural network measurements for synthetic data
df.head()

,label,timestamp,duration,pkg,dram,socket
0,Adult,1.653231e+09,9983915.060,141723026.0,25520553.0,0
1,Adult,1.653231e+09,9737717.851,132242459.0,23186400.0,0
2,Adult,1.653231e+09,9655013.949,130387117.0,23123655.0,0
3,Adult,1.653231e+09,9794125.674,132117520.0,24061789.0,0
4,Adult,1.653231e+09,9847082.163,133697046.0,24281498.0,0


In [2]:
# convert to seconds and Joules
df['duration'] *= (10**-6)
df['pkg'] *= (10**-6)
df['dram'] *= (10**-6)

duration = df['duration'].mean()
pkg = df['pkg'].mean()
dram = df['dram'].mean()

print(f"Average duration : {round(duration, 3)} in seconds")
print(f"Average pkg: {round(pkg, 3)} in Joules")
print(f"Average dram: {round(dram, 3)} in Joules")
print(f"\n Total energy used: {round(pkg+dram, 3)}")

Average duration : 6.548 in seconds
Average pkg: 91.907 in Joules
Average dram: 15.752 in Joules

 Total energy used: 107.659
